## Challenge 2: Video Game Sales Dataset Analysis

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("VideoGame").getOrCreate()

file_path = "/data/vgsales-12-4-2019-short.csv"
df = spark.read.option("sep", ",").option("header", "True").option("inferSchema", "true").csv(file_path)

# Convertir la colonne 'Year' en type entier
df = df.withColumn("Year", F.col("Year").cast("int"))

df.show(5)

+----+--------------------+--------+-----------+--------+----------------+----------------+------------+----------+-------------+------------+--------+---------+--------+-----------+----+
|Rank|                Name|   Genre|ESRB_Rating|Platform|       Publisher|       Developer|Critic_Score|User_Score|Total_Shipped|Global_Sales|NA_Sales|PAL_Sales|JP_Sales|Other_Sales|Year|
+----+--------------------+--------+-----------+--------+----------------+----------------+------------+----------+-------------+------------+--------+---------+--------+-----------+----+
|   1|          Wii Sports|  Sports|          E|     Wii|        Nintendo|    Nintendo EAD|         7.7|      NULL|        82.86|        NULL|    NULL|     NULL|    NULL|       NULL|2006|
|   2|   Super Mario Bros.|Platform|       NULL|     NES|        Nintendo|    Nintendo EAD|        10.0|      NULL|        40.24|        NULL|    NULL|     NULL|    NULL|       NULL|1985|
|   3|      Mario Kart Wii|  Racing|          E|     Wii|   

### Basic Information Extraction:

##### Determine the number of games published by each publisher.


In [13]:
count_games_by_publiser = df.groupBy("Publisher") \
    .agg(F.count("Name").alias("Number_of_games")) \
    .sort(F.desc("Number_of_games"))

count_games_by_publiser.show()

+--------------------+---------------+
|           Publisher|Number_of_games|
+--------------------+---------------+
|             Unknown|           4891|
|                Sega|           2085|
|             Ubisoft|           1519|
|          Activision|           1519|
|     Electronic Arts|           1498|
|              Konami|           1495|
|Sony Computer Ent...|           1354|
|            Nintendo|           1351|
|           Microsoft|           1267|
|                 THQ|           1065|
|              Capcom|            964|
|               Atari|            828|
|           EA Sports|            757|
|         Square Enix|            576|
|              Bandai|            566|
|        Namco Bandai|            554|
|Acclaim Entertain...|            511|
|         Hudson Soft|            460|
|  Namco Bandai Games|            452|
|               Namco|            441|
+--------------------+---------------+
only showing top 20 rows



##### Count the number of games available for each platform.

In [15]:
count_games_by_platform = df.groupBy("Platform") \
    .agg(F.count("Name").alias("Number_of_games")) \
    .sort(F.desc("Number_of_games"))

count_games_by_platform.show()

+--------+---------------+
|Platform|Number_of_games|
+--------+---------------+
|      PC|          10978|
|     PS2|           3564|
|      DS|           3292|
|      PS|           2703|
|     XBL|           2115|
|     PSN|           2004|
|     PS3|           1870|
|     PSP|           1804|
|     PS4|           1755|
|    X360|           1701|
|     Wii|           1674|
|     GBA|           1658|
|      GB|           1601|
|    SNES|           1209|
|     3DS|           1192|
|      NS|           1186|
|     NES|           1105|
|    XOne|           1082|
|     PSV|           1046|
|     And|           1007|
+--------+---------------+
only showing top 20 rows



### Data Improvement:

##### Create a new column combining 'Total_Shipped' and 'Global_Sales'. Use 'Total_Shipped' when available; otherwise, use 'Global_Sales'.

In [17]:
# Créer une nouvelle colonne combinant 'Total_Shipped' et 'Global_Sales'
df = df.withColumn(
    "Combined_Sales",
    F.when(F.col("Total_Shipped").isNotNull(), F.col("Total_Shipped"))
    .otherwise(F.col("Global_Sales"))
)

df.select("Total_Shipped", "Global_Sales", "Combined_Sales").show(20)

+-------------+------------+--------------+
|Total_Shipped|Global_Sales|Combined_Sales|
+-------------+------------+--------------+
|        82.86|        NULL|         82.86|
|        40.24|        NULL|         40.24|
|        37.14|        NULL|         37.14|
|         36.6|        NULL|          36.6|
|        33.09|        NULL|         33.09|
|        31.38|        NULL|         31.38|
|         30.8|        NULL|          30.8|
|        30.26|        NULL|         30.26|
|        30.22|        NULL|         30.22|
|        30.01|        NULL|         30.01|
|        28.31|        NULL|         28.31|
|        28.02|        NULL|         28.02|
|         24.0|        NULL|          24.0|
|        23.96|        NULL|         23.96|
|         23.6|        NULL|          23.6|
|         23.1|        NULL|          23.1|
|        22.67|        NULL|         22.67|
|        21.13|        NULL|         21.13|
|        20.61|        NULL|         20.61|
|         NULL|       20.32|    

##### Remove the now redundant columns.

In [18]:
# Suppressiion des colonnes 'Total_Shipped' et 'Global_Sales'
df = df.drop("Total_Shipped", "Global_Sales")

df.show(5)

+----+--------------------+--------+-----------+--------+----------------+----------------+------------+----------+--------+---------+--------+-----------+----+--------------+
|Rank|                Name|   Genre|ESRB_Rating|Platform|       Publisher|       Developer|Critic_Score|User_Score|NA_Sales|PAL_Sales|JP_Sales|Other_Sales|Year|Combined_Sales|
+----+--------------------+--------+-----------+--------+----------------+----------------+------------+----------+--------+---------+--------+-----------+----+--------------+
|   1|          Wii Sports|  Sports|          E|     Wii|        Nintendo|    Nintendo EAD|         7.7|      NULL|    NULL|     NULL|    NULL|       NULL|2006|         82.86|
|   2|   Super Mario Bros.|Platform|       NULL|     NES|        Nintendo|    Nintendo EAD|        10.0|      NULL|    NULL|     NULL|    NULL|       NULL|1985|         40.24|
|   3|      Mario Kart Wii|  Racing|          E|     Wii|        Nintendo|    Nintendo EAD|         8.2|       9.1|    N

### Critic Score Analysis:

##### Analyze the average critic score by ESRB Rating. Exclude rows with missing ESRB Rating or Critic Score.      AND Present the data with scores rounded to two decimal places and sorted by the average score.

In [19]:
# Filtrer pour garder uniquement les valeurs non null pour ESRB_Rating et Critic_Score
filtered_df = df.filter((F.col("ESRB_Rating").isNotNull()) & (F.col("Critic_Score").isNotNull()))

In [20]:
avg_CriticScore_by_ESRB = filtered_df.groupBy("ESRB_Rating") \
    .agg(F.round(F.avg("Critic_Score"),2).alias("average Critic Score")) \
    .sort(F.desc("average Critic Score"))

avg_CriticScore_by_ESRB.show()

+-----------+--------------------+
|ESRB_Rating|average Critic Score|
+-----------+--------------------+
|         EC|                 8.5|
|         RP|                8.02|
|          M|                7.47|
|          T|                7.24|
|          E|                7.11|
|        E10|                7.09|
+-----------+--------------------+



In [ ]:
# Arrêt de la SparkSession
spark.stop()